# 多线程实现fits文件的抠图

In [1]:
import os
import threading
import numpy as np
import pandas as pd
import time

In [2]:
#实现世界坐标转化为像素坐标
def world2pix(hdu,ra,dec):
    import astropy.wcs as wcs
    del hdu[0].header['RADECSYS']
    hdu[0].header['RADESYSa']='ICRS'
    w = wcs.WCS(hdu[0].header, hdu)
    hdu.close()
    x, y = w.all_world2pix(ra, dec, 1)

    return int(x),int(y)

In [3]:
#抠图
def acquire_image(path,ra,dec):
    from astropy.io import fits 
    import numpy as np
    hdu= fits.open(path)
    data=hdu[0].data
    data=data.byteswap().newbyteorder()
    #转换坐标系
    x,y=world2pix(hdu,float(ra),float(dec))
    #截图
    try:
        data1=data[y-10:y+10,x-10:x+10]
    except ValueError as e:
        print("valueError")
    return data1

In [4]:
def make_image(url):
    if url:
        data=[]
        filepaths=url['filepaths']
        ra=url['ra']
        dec=url['dec']
        name=url['image_path']
        
        for filepath in filepaths:
            data.append(acquire_image(filepath,ra,dec))

            if os.path.exists(name):
                pass
            else:
                try:
                    np.save(name,data)
                    print("save the"+name)
                except ValueError as e:
                    print("valueError")
                    pass
                    #保存data,并删除原有图像　
#                     for filepath in filepaths:
#                         os.remove(filepath) 
        
        

In [5]:
def images_handle(sample_path,image_path,path):#sample_path数据地址，image_path图像保存地址，path图像数据存储地址
    file=pd.read_csv(sample_path)
    n=file.shape[0]
    urls=[]
    bland=['u','g','r','i','z']
    for i in range(6000,10001):
        url={}
        filepaths=[]
        ra=file.iloc[i]['ra']
        dec=file.iloc[i]['dec']
        for j in range(5):
            file_path=path+str(i)+'_'+bland[j]+'.fits.bz2'
            if os.path.exists(file_path):
                filepaths.append(file_path)
        if len(filepaths)==5:
            url['filepaths']=filepaths
            url['ra']=ra
            url['dec']=dec
            name=image_path+str(i)+".npy"
            url['image_path']=name
        urls.append(url)
    urls=iter(urls)
    return urls

In [6]:
thread_num = 20
timeout = 5
lock = threading.Lock()

def loop(urls):
    print('thread %s is running...' % threading.current_thread().name)

    while True:
        try:
            with lock:
                url = next(urls)
        except StopIteration:
            break
        try:
            make_image(url)
        except:
            print('exceptfail\t%s' % url)
    print('thread %s is end...' % threading.current_thread().name)


In [7]:
if __name__ == '__main__':
    sample_path = r"/home/qiao/文档/data/star.csv"
    image_path="/home/qiao/文档/data/star_image/"
    path="/home/qiao/文档/data/star_bz2/"
    urls = images_handle(sample_path,image_path,path)
#     for url in urls:
#         print(url)
    start=time.time()
    for i in range(0, thread_num):
        t = threading.Thread(target=loop, name='LoopThread %s' % i, args=(urls,))
        t.start()
    end=time.time()
    print(end-start)

thread LoopThread 0 is running...thread LoopThread 1 is running...
thread LoopThread 2 is running...

thread LoopThread 3 is running...
thread LoopThread 4 is running...
thread LoopThread 5 is running...
thread LoopThread 6 is running...
thread LoopThread 7 is running...
thread LoopThread 8 is running...
thread LoopThread 9 is running...
thread LoopThread 10 is running...
thread LoopThread 11 is running...
thread LoopThread 12 is running...
thread LoopThread 13 is running...
thread LoopThread 14 is running...
thread LoopThread 15 is running...
thread LoopThread 16 is running...
thread LoopThread 17 is running...
thread LoopThread 18 is running...
thread LoopThread 19 is running...
0.010686159133911133
save the/home/qiao/文档/data/star_image/6316.npy
save the/home/qiao/文档/data/star_image/6241.npy
save the/home/qiao/文档/data/star_image/6249.npy
save the/home/qiao/文档/data/star_image/6239.npy
save the/home/qiao/文档/data/star_image/6207.npy
save the/home/qiao/文档/data/star_image/6319.npy
save th

save the/home/qiao/文档/data/star_image/6480.npy
save the/home/qiao/文档/data/star_image/6481.npy
save the/home/qiao/文档/data/star_image/6486.npy
save the/home/qiao/文档/data/star_image/6482.npy
save the/home/qiao/文档/data/star_image/6483.npy
save the/home/qiao/文档/data/star_image/6484.npy
save the/home/qiao/文档/data/star_image/6493.npy
save the/home/qiao/文档/data/star_image/6487.npy
save the/home/qiao/文档/data/star_image/6489.npy
save the/home/qiao/文档/data/star_image/6488.npy
save the/home/qiao/文档/data/star_image/6496.npy
save the/home/qiao/文档/data/star_image/6494.npy
save the/home/qiao/文档/data/star_image/6495.npy
save the/home/qiao/文档/data/star_image/6497.npy
save the/home/qiao/文档/data/star_image/6498.npy
save the/home/qiao/文档/data/star_image/6500.npy
save the/home/qiao/文档/data/star_image/6499.npy
save the/home/qiao/文档/data/star_image/6501.npy
save the/home/qiao/文档/data/star_image/6502.npy
save the/home/qiao/文档/data/star_image/6503.npy
save the/home/qiao/文档/data/star_image/6504.npy
save the/home

save the/home/qiao/文档/data/star_image/6671.npy
save the/home/qiao/文档/data/star_image/6672.npy
save the/home/qiao/文档/data/star_image/6673.npy
save the/home/qiao/文档/data/star_image/6674.npy
save the/home/qiao/文档/data/star_image/6675.npy
save the/home/qiao/文档/data/star_image/6677.npy
save the/home/qiao/文档/data/star_image/6676.npy
save the/home/qiao/文档/data/star_image/6681.npy
save the/home/qiao/文档/data/star_image/6678.npy
save the/home/qiao/文档/data/star_image/6680.npy
save the/home/qiao/文档/data/star_image/6683.npy
save the/home/qiao/文档/data/star_image/6684.npy
save the/home/qiao/文档/data/star_image/6682.npy
save the/home/qiao/文档/data/star_image/6686.npy
save the/home/qiao/文档/data/star_image/6685.npy
save the/home/qiao/文档/data/star_image/6687.npy
save the/home/qiao/文档/data/star_image/6688.npy
save the/home/qiao/文档/data/star_image/6689.npy
save the/home/qiao/文档/data/star_image/6690.npy
save the/home/qiao/文档/data/star_image/6691.npy
save the/home/qiao/文档/data/star_image/6693.npy
save the/home

save the/home/qiao/文档/data/star_image/6724.npy
save the/home/qiao/文档/data/star_image/6727.npy
save the/home/qiao/文档/data/star_image/6726.npy
save the/home/qiao/文档/data/star_image/6728.npysave the/home/qiao/文档/data/star_image/6725.npy

save the/home/qiao/文档/data/star_image/6731.npy
save the/home/qiao/文档/data/star_image/6729.npy
save the/home/qiao/文档/data/star_image/6730.npy
save the/home/qiao/文档/data/star_image/6733.npy
save the/home/qiao/文档/data/star_image/6732.npy
save the/home/qiao/文档/data/star_image/6734.npy
save the/home/qiao/文档/data/star_image/6735.npy
save the/home/qiao/文档/data/star_image/6737.npy
save the/home/qiao/文档/data/star_image/6736.npy
save the/home/qiao/文档/data/star_image/6738.npy
save the/home/qiao/文档/data/star_image/6739.npy
save the/home/qiao/文档/data/star_image/6740.npy
save the/home/qiao/文档/data/star_image/6741.npy
save the/home/qiao/文档/data/star_image/6742.npy
save the/home/qiao/文档/data/star_image/6744.npy
save the/home/qiao/文档/data/star_image/6748.npy
save the/home

save the/home/qiao/文档/data/star_image/6921.npy
save the/home/qiao/文档/data/star_image/6922.npy
save the/home/qiao/文档/data/star_image/6923.npy
save the/home/qiao/文档/data/star_image/6925.npy
save the/home/qiao/文档/data/star_image/6927.npy
save the/home/qiao/文档/data/star_image/6926.npy
save the/home/qiao/文档/data/star_image/6928.npy
save the/home/qiao/文档/data/star_image/6929.npy
save the/home/qiao/文档/data/star_image/6932.npy
save the/home/qiao/文档/data/star_image/6931.npy
save the/home/qiao/文档/data/star_image/6933.npy
save the/home/qiao/文档/data/star_image/6934.npy
save the/home/qiao/文档/data/star_image/6937.npy
save the/home/qiao/文档/data/star_image/6936.npy
save the/home/qiao/文档/data/star_image/6935.npy
save the/home/qiao/文档/data/star_image/6939.npy
save the/home/qiao/文档/data/star_image/6940.npy
save the/home/qiao/文档/data/star_image/6941.npy
save the/home/qiao/文档/data/star_image/6943.npy
save the/home/qiao/文档/data/star_image/6944.npy
save the/home/qiao/文档/data/star_image/6945.npy
save the/home

...

save the/home/qiao/文档/data/star_image/8917.npy
save the/home/qiao/文档/data/star_image/8918.npy
save the/home/qiao/文档/data/star_image/8919.npy
save the/home/qiao/文档/data/star_image/8920.npy
save the/home/qiao/文档/data/star_image/8921.npy
save the/home/qiao/文档/data/star_image/8922.npy
save the/home/qiao/文档/data/star_image/8923.npy
save the/home/qiao/文档/data/star_image/8924.npy
save the/home/qiao/文档/data/star_image/8925.npy
save the/home/qiao/文档/data/star_image/8926.npy
save the/home/qiao/文档/data/star_image/8928.npy
save the/home/qiao/文档/data/star_image/8929.npy
save the/home/qiao/文档/data/star_image/8930.npy
save the/home/qiao/文档/data/star_image/8931.npy
save the/home/qiao/文档/data/star_image/8932.npy
save the/home/qiao/文档/data/star_image/8933.npy
save the/home/qiao/文档/data/star_image/8934.npy
save the/home/qiao/文档/data/star_image/8935.npy
save the/home/qiao/文档/data/star_image/8936.npy
save the/home/qiao/文档/data/star_image/8937.npy
save the/home/qiao/文档/data/star_image/8938.npy
save the/home

save the/home/qiao/文档/data/star_image/9122.npy
save the/home/qiao/文档/data/star_image/9123.npy
save the/home/qiao/文档/data/star_image/9125.npy
save the/home/qiao/文档/data/star_image/9126.npy
save the/home/qiao/文档/data/star_image/9127.npy
save the/home/qiao/文档/data/star_image/9129.npy
save the/home/qiao/文档/data/star_image/9130.npy
save the/home/qiao/文档/data/star_image/9131.npy
save the/home/qiao/文档/data/star_image/9132.npy
save the/home/qiao/文档/data/star_image/9133.npy
save the/home/qiao/文档/data/star_image/9134.npy
save the/home/qiao/文档/data/star_image/9135.npy
save the/home/qiao/文档/data/star_image/9136.npy
save the/home/qiao/文档/data/star_image/9138.npy
save the/home/qiao/文档/data/star_image/9137.npy
save the/home/qiao/文档/data/star_image/9139.npy
save the/home/qiao/文档/data/star_image/9140.npy
save the/home/qiao/文档/data/star_image/9142.npy
save the/home/qiao/文档/data/star_image/9143.npy
save the/home/qiao/文档/data/star_image/9144.npy
save the/home/qiao/文档/data/star_image/9145.npy
save the/home

save the/home/qiao/文档/data/star_image/9383.npy
save the/home/qiao/文档/data/star_image/9384.npy
save the/home/qiao/文档/data/star_image/9385.npy
save the/home/qiao/文档/data/star_image/9387.npy
save the/home/qiao/文档/data/star_image/9389.npy
save the/home/qiao/文档/data/star_image/9388.npy
save the/home/qiao/文档/data/star_image/9396.npy
save the/home/qiao/文档/data/star_image/9401.npy
save the/home/qiao/文档/data/star_image/9407.npysave the/home/qiao/文档/data/star_image/9403.npy

save the/home/qiao/文档/data/star_image/9410.npy
save the/home/qiao/文档/data/star_image/9413.npy
save the/home/qiao/文档/data/star_image/9412.npy
save the/home/qiao/文档/data/star_image/9414.npy
save the/home/qiao/文档/data/star_image/9415.npy
save the/home/qiao/文档/data/star_image/9416.npy
save the/home/qiao/文档/data/star_image/9417.npy
save the/home/qiao/文档/data/star_image/9418.npy
save the/home/qiao/文档/data/star_image/9419.npy
save the/home/qiao/文档/data/star_image/9420.npy
save the/home/qiao/文档/data/star_image/9422.npy
save the/home

save the/home/qiao/文档/data/star_image/9617.npy
save the/home/qiao/文档/data/star_image/9618.npy
save the/home/qiao/文档/data/star_image/9619.npy
save the/home/qiao/文档/data/star_image/9620.npy
save the/home/qiao/文档/data/star_image/9621.npy
save the/home/qiao/文档/data/star_image/9622.npy
save the/home/qiao/文档/data/star_image/9624.npy
save the/home/qiao/文档/data/star_image/9623.npy
save the/home/qiao/文档/data/star_image/9625.npy
save the/home/qiao/文档/data/star_image/9626.npy
save the/home/qiao/文档/data/star_image/9627.npy
save the/home/qiao/文档/data/star_image/9628.npy
save the/home/qiao/文档/data/star_image/9629.npy
save the/home/qiao/文档/data/star_image/9630.npy
save the/home/qiao/文档/data/star_image/9631.npy
save the/home/qiao/文档/data/star_image/9632.npy
save the/home/qiao/文档/data/star_image/9633.npy
save the/home/qiao/文档/data/star_image/9634.npy
save the/home/qiao/文档/data/star_image/9635.npy
save the/home/qiao/文档/data/star_image/9636.npy
save the/home/qiao/文档/data/star_image/9637.npy
save the/home

save the/home/qiao/文档/data/star_image/9822.npy
save the/home/qiao/文档/data/star_image/9824.npy
save the/home/qiao/文档/data/star_image/9825.npy
save the/home/qiao/文档/data/star_image/9826.npy
save the/home/qiao/文档/data/star_image/9827.npy
save the/home/qiao/文档/data/star_image/9828.npy
save the/home/qiao/文档/data/star_image/9829.npy
save the/home/qiao/文档/data/star_image/9830.npy
save the/home/qiao/文档/data/star_image/9831.npy
save the/home/qiao/文档/data/star_image/9832.npy
save the/home/qiao/文档/data/star_image/9834.npy
save the/home/qiao/文档/data/star_image/9833.npy
save the/home/qiao/文档/data/star_image/9835.npy
save the/home/qiao/文档/data/star_image/9836.npy
save the/home/qiao/文档/data/star_image/9837.npy
save the/home/qiao/文档/data/star_image/9838.npy
save the/home/qiao/文档/data/star_image/9840.npy
save the/home/qiao/文档/data/star_image/9844.npy
save the/home/qiao/文档/data/star_image/9845.npy
save the/home/qiao/文档/data/star_image/9847.npy
save the/home/qiao/文档/data/star_image/9848.npy
save the/home